In [ ]:
# ok so activation function gives non linearity and most visualization of this is that
# it saves you from the computational overhead like if problem is positive multiply by 2 than by using relu we don't multiply with 2 if negative only gives 0 output

#so positional embedding only added once at beggining in transformer

# embedding store information like simillar meaning words will have more values in vectors
# so attention can store info like verb-object,subject and maybe overall meaning  these three will require three attention head
# positional encoding from the name we can say position of words like it's gonna be messed up if position of words are changed
# example = i killed a earthworm but if model gets earthworm killed i that's gonna be messed up
# positional encodings are learnable parameter
# everything is a different learnable parameter in transformer
#scale.product in attention is for gpu we can scale in gpu
# and yes we mayb use attention and other stuff on the first encoder because  in first encoder may loose information that's why a lot of informatuion
# will be lost damn attention also reweights in first encoder
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#2 hyperparameter
#bock for length interger (interger maybe encoding stuff)
block_size= 64
batch_size=128
#batch for how many in the parallel
max_iters= 3000
#eval_interval = 2500
#we can think momentum and learning rate in action by thinking that the model is learning
# as a car is coming downhill and it's pedal press is learning rate and momentum from coming down
# is momentum
learning_rate = 3e-4
#maybe for the after number of iterations we want to print loss to know how training
# is going and it's even learning or not
eval_iters = 100
dropout = 0.2
#maybe number of encoder decoder layer
n_layer = 8
n_embd =384 #embedding length
#so according to n_emb each embeddingvector = [1,2,3..........384]
n_head = 8

cuda


In [ ]:
chars = ''
with open('wizard_of_oz.txt' ,'r',encoding ='utf-8') as f:
    text=f.read()
#print(text[:200])
    chars = sorted(list(set(text)))
print(chars)
print(len(chars))
vocabulary_size= len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [ ]:
#for the encoder part
# ok so string to int we are making dictionary so ch:i so ch is key and i is value
#just like a encoded
#encoding word to number since nn only understands the number
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: [int_to_string[i] for i in l]
print(encode('hello'))
decodehello = decode(encode('hello'))
print(decodehello)

[61, 58, 65, 65, 68]
['h', 'e', 'l', 'l', 'o']


In [ ]:
# oh so tensor sayad kuch nyi chij hogi iske andr and new features that's why we used tensor to
#store encoded data
data = torch.tensor(encode(text),dtype = torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [ ]:
#so here i am splitting data
#maybe if we did direct then it would have multiplied the every elements with 0.8
trainingdatalen = int(0.8*len(data))
train_data= data[:trainingdatalen]
val_data=data[trainingdatalen:]
def getbatch(split):
  data=train_data if split=='train' else val_data
  #i think it's just taking simple random for each batch
  #random indices from which we can start generating text
  ix = torch.randint(len(data)-block_size,(batch_size,))
  #print(ix)
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y
x,y = getbatch('train')
#input
#print(x)
#target
#print(y)

In [ ]:
@torch.no_grad()
def estimate_loss():
  out={}
  #so dropout is set to off while evaluation of the model
  model.eval()
  for split in ['train','val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y =getbatch(split)
      logits,loss = model(X,Y)
      losses[k]= loss.item()
    out[split]=losses.mean()
  model.train()
  return out

In [ ]:
#so here maybe we are doing which character will come next
#bigram a block then slide window maybe
# x=trainingdatalen[:block_size]
# y=trainingdatalen[1:block_size+1]
# for t in range(block_size):
#     context =x[:t]
#     target=y[t]
#     print(f"when input is {context} target is {target}")

In [ ]:
class Head(nn.Module):
  """ one head of self-attention """
  def __init__(self,heads_size):
    super().__init__()
    self.key =nn.Linear(n_embd,heads_size,bias=False)
    self.query =nn.Linear(n_embd,heads_size,bias=False)
    self.value =nn.Linear(n_embd,heads_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.dropout= nn.Dropout(dropout)
  def forward(self,x):
    # input of size (batch, time-step, channels)
    # output of size (batch, time-step, head size)
    B,T,C = x.shape
    k = self.key(x) #(B,T,hs)
    q = self.query(x) #(B,T,hs)
    # compute attention scores ("affinities")
    wei = q @ k.transpose(-2,-1) *k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) # (B, T, T)
    wei = F.softmax(wei,dim=-1)
    wei =self.dropout(wei)
    v = self.value(x)# (B,T,hs)
    out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
    return out

class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(head_size* num_heads,n_embd)
    self.dropout = nn.Dropout(dropout)
  def forward(self,x):                                 # batch,time,channel and these multiple h1 are just output after calculating attention using q,k and v
    out = torch.cat([h(x) for h in self.heads],dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  def __init__(self,n_embd):
   super().__init__()
   self.net =nn.Sequential(nn.Linear(n_embd,4*n_embd), # as the name says linear so just implies linear function (regression )and
   #and increase the diamensions of the vector
   nn.ReLU(),
   #yes it has 2 layers of ffn inside a big block of network
   nn.Linear(4*n_embd,n_embd),
   nn.Dropout(dropout)
   )
  def forward(self,x):
    return self.net(x)

class Block(nn.Module):
  """ Transformer block: communication followed by computation """
  def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
  def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
#nn.module are learnable parameter
##ok so in reality nn.Module is a base class
# so unnormalized prediction of model is logist maybe
class GPTLanguageModel(nn.Module):
 def __init__(self,vocabulary_size):
    super().__init__()
    #embedding stores information about char or word
    #embedding like in characters may be a comes first so bigger number than
    #b and in words maybe king and man so king will have more larger number
    #
    self.token_embedding_table =nn.Embedding(vocabulary_size,n_embd)
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    #since sequential decoder layers
    self.blocks=nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd,vocabulary_size)
 def _init_weights(self,module):
  if isinstance(module,nn.Linear):
    torch.nn.init.normal_(module.weight,mean=0.0,std =0.2)
    if module.bias is not None:
      torch.nn.init.zeros_(module.bias)
    elif isinstance(module,nn.Embedding):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.2)
 def forward(self,index,target=None):
      B, T = index.shape

      #idx and targets are both (B,T) tensors of integers
      tok_emb = self.token_embedding_table(index) #(B,T,C)
      pos_emb = self.position_embedding_table(torch.arange(T,device = device)) #(T,C)
      x=tok_emb+pos_emb#(B,T,C)
      x=self.blocks(x)#(B,T,C)
      x=self.ln_f(x)#(B,T,C)
      logits = self.lm_head(x)
      if target is None:
        loss = None
      else:
       B, T, C = logits.shape
       logits = logits.view(B*T,C)
       #so we did reshape here because of F.cross_entropy funtion takes
       #since there are multiple ways of giving it parameters since our
       # logits.view gives B,T and C
       targets=target.view(B*T)
       loss = F.cross_entropy(logits,targets)
      return logits,loss
#generate function is gonna generate the tokens
# ok so generate wraps up the logits of forward pass than to travel back in time and then feed that to forward pass agaian
 def generate(self,index,max_new_tokens):
    for _ in range(max_new_tokens):
      #get the predictions
      logits,loss = self.forward(index)
      #focus only on the last time step
      logits = logits[:,-1,:] #become B,c
      # apply softmax function to get probabilities
      probs = F.softmax(logits,dim=-1)
      #sample from the distribution
      index_next =torch.multinomial(probs,num_samples = 1)
      index = torch.cat((index,index_next),dim=1)
    return index


model = GPTLanguageModel(vocabulary_size)
m = model.to(device)

In [ ]:
#create a pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

for iter in range(max_iters):
  if iter % eval_iters ==0:
    losses = estimate_loss()
    print(f'Round: {iter} = train losses = {losses["train"]:.3f} val losses = {losses["val"]:.3f}')
  #sample a batch of data
  xb,yb= getbatch('train')

  #evaluate the loss
  logits,loss =model(xb,yb)
  # so that it don't add up the previous gradient sometime data is weir
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(f'{loss.item():.3f}')


Round: 0 = train losses = 4.517 val losses = 4.510
Round: 100 = train losses = 2.278 val losses = 2.351
Round: 200 = train losses = 1.903 val losses = 2.014
Round: 300 = train losses = 1.647 val losses = 1.805
Round: 400 = train losses = 1.481 val losses = 1.680
Round: 500 = train losses = 1.380 val losses = 1.608
Round: 600 = train losses = 1.292 val losses = 1.571
Round: 700 = train losses = 1.228 val losses = 1.538
Round: 800 = train losses = 1.170 val losses = 1.520
Round: 900 = train losses = 1.110 val losses = 1.498
Round: 1000 = train losses = 1.071 val losses = 1.511
Round: 1100 = train losses = 1.017 val losses = 1.496
Round: 1200 = train losses = 0.970 val losses = 1.523
Round: 1300 = train losses = 0.930 val losses = 1.527
Round: 1400 = train losses = 0.881 val losses = 1.551
Round: 1500 = train losses = 0.839 val losses = 1.562
Round: 1600 = train losses = 0.799 val losses = 1.584
Round: 1700 = train losses = 0.749 val losses = 1.613


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "wizard_gpt.pth")
print("Model saved as wizard_gpt.pth")

Model saved as wizard_gpt.pth
